In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('./output/thrombolysis_choice_benefit_he.csv')
data = data[data['infarction'] ==1]
data['all'] = 1
list(data)

['Unnamed: 0',
 'stroke_team',
 'male',
 'year',
 'arrive_by_ambulance',
 'onset_to_arrival_time',
 'onset_during_sleep',
 'arrival_to_scan_time',
 'onset_to_scan',
 'infarction',
 'stroke_severity',
 'precise_onset_known',
 'prior_disability',
 'any_afib_diagnosis',
 'afib_anticoagulant',
 'age',
 'onset_to_thrombolysis',
 'thrombolysis',
 'thrombectomy',
 'scan_to_thrombolysis_time',
 'discharge_disability',
 'simulated_onset_to_thrombolysis',
 'thrombolysis_choice_probability',
 'thrombolysis_choice',
 'hospital_SHAP',
 'benchmark_decision',
 'untreated_weighted_mrs',
 'untreated_0_to_4',
 'untreated_mrs_0',
 'untreated_mrs_1',
 'untreated_mrs_2',
 'untreated_mrs_3',
 'untreated_mrs_4',
 'untreated_mrs_5',
 'untreated_mrs_6',
 'untreated_utility',
 'treated_weighted_mrs',
 'treated_0_to_4',
 'treated_mrs_0',
 'treated_mrs_1',
 'treated_mrs_2',
 'treated_mrs_3',
 'treated_mrs_4',
 'treated_mrs_5',
 'treated_mrs_6',
 'treated_utility',
 'change_in_weighted_mrs',
 'change_in_mrs_0_to_4

In [3]:
# Limit to no afib
mask = data['afib_anticoagulant'] == 0
data = data[mask]

proportion = 1 - mask.mean()
print (f'Proportion of patients receiving anticoagulants: {proportion:.2f}')

Proportion of patients receiving anticoagulants: 0.13


In [4]:
# Set dataframe dispplay rows to 100
pd.set_option('display.max_rows', 100)

data.head().T

,0,2,4,6,7
Unnamed: 0,0,2,5,7,8
stroke_team,Royal Berkshire Hospital,Royal Berkshire Hospital,Queen Elizabeth Hospital Edgbaston,Wycombe General Hospital,Queen's Medical Centre - Nottingham
male,0,1,0,1,1
year,2017,2017,2016,2021,2020
arrive_by_ambulance,1.0,1.0,1.0,1.0,1.0
onset_to_arrival_time,65.0,190.0,111.0,96.0,138.0
onset_during_sleep,0,0,0,0,0
arrival_to_scan_time,5.0,15.0,39.0,6.0,52.0
onset_to_scan,70.0,205.0,150.0,102.0,190.0
infarction,1.0,1.0,1.0,1.0,1.0


In [5]:
# Set HE columns when dead

mask = data['used_untreated_mrs'] == 6
data.loc[mask, 'untreated_life_expectancy'] = data.loc[mask, 'age']
data.loc[mask, 'untreated_qalys_total'] = 0
data.loc[mask, 'untreated_care_years'] = 0
data.loc[mask, 'untreated_survival_median_years'] = 0
data.loc[mask, 'untreated_total_discounted_cost'] = 0

mask = data['used_treated_mrs'] == 6
data.loc[mask, 'treated_life_expectancy'] = data.loc[mask, 'age']
data.loc[mask, 'treated_qalys_total'] = 0
data.loc[mask, 'treated_care_years'] = 0
data.loc[mask, 'treated_survival_median_years'] = 0
data.loc[mask, 'treated_total_discounted_cost'] = 0


In [6]:
results = pd.DataFrame()

# Get descriptive statistics for untreated patients
untreated_cols = [
    'untreated_survival_median_years',
    'untreated_life_expectancy',
    'untreated_care_years',
    'untreated_qalys_total',
    'untreated_life_expectancy',
    'untreated_total_discounted_cost',
    'untreated_mrs_0-2',
    'untreated_mrs_5-6']

treated_cols = [
    'treated_survival_median_years',
    'treated_life_expectancy',
    'treated_care_years',
    'treated_qalys_total',
    'treated_life_expectancy',
    'treated_total_discounted_cost',
    'treated_mrs_0-2',
    'treated_mrs_5-6']

scenarios = ['all', 'thrombolysis', 'benchmark_decision', 'improved_outcome']

for scenario in scenarios:
    mask = data[scenario] == 1
    selection_untreated = data.loc[mask][untreated_cols].describe().T
    results[f'{scenario}_untreated'] = selection_untreated['mean']
    selection_treated = data.loc[mask][treated_cols].describe().T
    results[f'{scenario}_treated'] = selection_treated['mean'].values
    results[f'{scenario}_diff'] = results[f'{scenario}_treated'] - results[f'{scenario}_untreated']



results.T.to_csv('./output/he_summary.csv')
results.T.round(3)

,untreated_survival_median_years,untreated_life_expectancy,untreated_care_years,untreated_qalys_total,untreated_life_expectancy,untreated_total_discounted_cost,untreated_mrs_0-2,untreated_mrs_5-6
all_untreated,7.603,82.033,0.281,5.016,82.033,20378.755,0.527,0.198
all_treated,7.895,82.324,0.252,5.245,82.324,19680.462,0.556,0.177
all_diff,0.292,0.291,-0.029,0.229,0.291,-698.293,0.028,-0.021
thrombolysis_untreated,7.609,81.987,0.281,5.023,81.987,20404.967,0.475,0.235
thrombolysis_treated,7.910,82.269,0.255,5.258,82.269,19817.153,0.543,0.188
thrombolysis_diff,0.301,0.282,-0.027,0.236,0.282,-587.814,0.068,-0.046
benchmark_decision_untreated,7.631,82.003,0.280,5.038,82.003,20393.844,0.466,0.240
benchmark_decision_treated,7.930,82.291,0.254,5.272,82.291,19790.849,0.535,0.193
benchmark_decision_diff,0.299,0.288,-0.027,0.234,0.288,-602.995,0.069,-0.047
improved_outcome_untreated,7.480,82.117,0.275,4.936,82.117,19997.996,0.391,0.301
